# Attention Is All You Need

__References:__
* Paper: [Attention is all you need](https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf)
* Blog: [VANDERGOTEN.ai](http://vandergoten.ai/2018-09-18-attention-is-all-you-need/)



__FCNNs:__

[-] Fully-connected layers do not account for `sequential information`.

__RNNs:__

[+] It can handle `dynamic sequence lengths`.

[-] But it has `vanishing gradient problem`.

[-] Because of `iterative nature`, it takes quite a `long time` for information to propagate (linear in the length of the sequence).

__CNNs:__

[+] It `immune` to the problem of `long time` because here each `kernel` application is performed in `isolation` (within the same layer). 

[-] Unfortunately this comes at the `cost of` only being able to account for `fixed-sized contexts`.

__Transformer:__

[+] It leverages fully-connected networks with a `cleverly-designed attention mechanism`. 

[+] This enables us to ``circumvent(to bypass) interdependent computations`` that RNNs are facing.

__Seq2Seq using RNNs:__

[-] It needs to pack all of the information of the input sequence into a single vector.

[-] Since the input sequences’ `lengths are unbounded`, we have to put an `unbounded amount of information` into single vector.

[-] As the `vector’s capacity is bounded`, it becomes `impossible` for the `decoder` to produce sensible output sequences of `arbitrary length`.

# The Illustrated Transformer

by [Jay Alammar](http://jalammar.github.io/illustrated-transformer/)

## Transformer as a Black-Box 

<img src = "http://jalammar.github.io/images/t/the_transformer_3.png" width = "70%">

                                           For Machine Translation Application

## Two Components (Encoding-Decoding)

<img src = "http://jalammar.github.io/images/t/The_transformer_encoders_decoders.png" width = "50%">

<img src = "https://cdn-images-1.medium.com/freeze/max/1000/1*0K0LqqhZfdw9qD7i3a2w_A.png?q=20" width = "50%">



## Stacked Encoder and Decoder

The encoding & decoding components are stack of 6 encoders and decoders resp.

<img src = "http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" width = "70%">

## Inside of Single Encoder

The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:

<img src = "http://jalammar.github.io/images/t/Transformer_encoder.png" width = "60%">

__Self-Attention:__ a layer that helps the encoder look at other words in the input sentence as it encodes a specific word.

__FFNN:__ The exact same feed-forward network is independently applied to each position.


## Inside of Single Decoder

The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).

<img src = "http://jalammar.github.io/images/t/Transformer_decoder.png" width = "70%">


## Vectors/Tensors flows between components

`Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.`
<img src = "http://jalammar.github.io/images/t/embeddings.png" width = "70%">

* The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512.

* In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that’s directly below.

## Flow of word embeddings between Encoders

After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.

<img src = "http://jalammar.github.io/images/t/encoder_with_tensors.png" width = "50%">

* The word in each position flows through its own path in the encoder. 
* There are dependencies between these paths in the self-attention layer. 
* The feed-forward layer does not have those dependencies, 
* Thus the various paths can be executed in parallel while flowing through the feed-forward layer.

# Understand Each Sub-Layer via Example


# Encoder

* An encoder receives a list of vectors as input. 
* It processes this list by passing these vectors into a ‘self-attention’ layer.
* Then into a FFNN, 
* Then sends out the output upwards to the next encoder.

<img src = "http://jalammar.github.io/images/t/encoder_with_tensors_2.png" width = "60%">
                
                            The exact same network with each vector flowing through it separately.

## Self-Attention at a High Level

Say the following sentence is an input sentence we want to translate:

``` diff 
- ”The animal didn't cross the street because it was too tired” 
```

What does “it” in this sentence refer to? Is it referring to the street or to the animal? It’s a simple question to a human, but not as simple to an algorithm.

<img src = "http://jalammar.github.io/images/t/transformer_self-attention_visualization.png" width = "40%">


## Self-Attention in Detail

__First step__ in calculating self-attention is to __create three vectors__ from each of the encoder’s input vectors (in this case, the embedding of each word). 

So for each word, we create,
1. Query vector, 
2. Key vector, 
3. Value vector.

Notice that these new vectors are `smaller in dimension` than the embedding vector. Their dimensionality is 64, while the embedding and encoder input/output vectors have dimensionality of 512. They don’t HAVE to be smaller, this is an architecture choice to make the computation of `multiheaded attention` (mostly) constant.

<img src = "http://jalammar.github.io/images/t/transformer_self_attention_vectors.png" width = "60%">

Multiplying x1 by the WQ weight matrix produces q1, the "query" vector associated with that word. We end up creating a "query", a "key", and a "value" projection of each word in the input sentence.

__Second Step__ in calculating __self-attention__ is to calculate a score. 

Say we’re calculating the self-attention for the first word in this example, `“Thinking”`. We need to score each word of the input sentence against this word. 

_The __score__ determines how much focus to place on other parts of the input sentence as we encode a word at a certain position._

The score is calculated by taking the `dot product` of the `query vector` with the `key vector` of the respective word we’re scoring. 

So if we’re processing the self-attention for the word in position #1, the first score would be the dot product of q1 and k1. The second score would be the dot product of q1 and k2.

<img src = "http://jalammar.github.io/images/t/transformer_self_attention_score.png" width = "70%">

__Third step__ is to divide the scores by 8 (the __square root of the dimension of the key vectors__ used in the paper – 64. This leads to having more stable gradients. There could be other possible values here, but this is the default).

__Forth Step__ is to pass the result through a __softmax operation__. Softmax normalizes the scores so they’re all positive and add up to 1.

<img src = "http://jalammar.github.io/images/t/self-attention_softmax.png" width = "70%">

This softmax score determines how much each word will be expressed at this position. Clearly the word at this position will have the highest softmax score, `but sometimes it’s useful to attend to another word that is relevant to the current word.`

The __fifth step__ is to __multiply each value vector by the softmax score__ (in preparation to sum them up). The intuition here is to keep intact the values of the word(s) we want to focus on, and drown-out irrelevant words (by multiplying them by tiny numbers like 0.001, for example).

The __sixth step__ is to __sum up the weighted value vectors__. This produces the output of the self-attention layer at this position (for the first word).

<img src = "http://jalammar.github.io/images/t/self-attention-output.png" width = "70%">

That concludes the self-attention calculation. The resulting vector is one we can send along to the feed-forward neural network. In the actual implementation, however, this calculation is done in matrix form for faster processing. So let’s look at that now that we’ve seen the intuition of the calculation on the word level.


## Matrix Calculation of Self-Attention

The __first step__ is to calculate the Query, Key, and Value matrices. We do that by packing our embeddings into a matrix X, and multiplying it by the weight matrices we’ve trained (WQ, WK, WV).

<img src = "http://jalammar.github.io/images/t/self-attention-matrix-calculation.png" width = "50%">

`Every row in the X matrix corresponds to a word in the input sentence. We again see the difference in size of the embedding vector (512, or 4 boxes in the figure), and the q/k/v vectors (64, or 3 boxes in the figure)`

__Finally__, since we’re dealing with matrices, we can condense steps two through six in one formula to calculate the outputs of the self-attention layer.

<img src = "http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" width = "60%">

`The self-attention calculation in matrix form`


## The Beast With Many Heads

__“Multi-Headed” Attention__ improves the performance of the attention layer in two ways:

1. It expands the model’s ability to `focus on different positions`. Yes, in the example above, z1 contains a little bit of every other encoding, but it could be dominated by the the actual word itself. It would be useful if we’re translating a sentence like `“The animal didn’t cross the street because it was too tired”`, we would want to know which word `“it”` refers to.

2. It gives the attention layer multiple `“representation subspaces”`. With multi-headed attention we have not only one, but `multiple sets of Query/Key/Value weight matrices` (the Transformer uses `eight attention heads`, so we end up with `eight sets for each encoder/decoder`). Each of these sets is `randomly initialized`. Then, after training, each set is used to project the input embeddings (or vectors from lower encoders/decoders) into a different representation subspace.

<img src = "http://jalammar.github.io/images/t/transformer_attention_heads_qkv.png" width = "60%">

`With multi-headed attention, we maintain separate Q/K/V weight matrices for each head resulting in different Q/K/V matrices. As we did before, we multiply X by the WQ/WK/WV matrices to produce Q/K/V matrices.`

If we do the same self-attention calculation we outlined above, just eight different times with different weight matrices, we end up with `eight different Z matrices`.

<img src = "http://jalammar.github.io/images/t/transformer_attention_heads_z.png" width = "60%">

This leaves us with a bit of a challenge. The feed-forward layer is not expecting eight matrices – it’s expecting a single matrix (a vector for each word). So we need a way to `condense these eight down into a single matrix`.

How do we do that? We concat the matrices then multiple them by an additional weights matrix WO.

<img src = "http://jalammar.github.io/images/t/transformer_attention_heads_weight_matrix_o.png" width = "60%">


## All steps in Single Place

<img src = "http://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png" width = "90%">


## Different Attentions in Attention Layers

Example where the different attention heads are focusing as we encode the word “it” in our example sentence:

<img src = "http://jalammar.github.io/images/t/transformer_self-attention_visualization_2.png" width = "40%">

As we encode the word `"it"`, __one attention head__ is focusing most on `"the animal"`, while __another is focusing__ on `"tired"` -- in a sense, the model's representation of the word `"it"` bakes in some of the representation of both `"animal"` and `"tired"`.

If we add all the attention heads to the picture, however, things can be harder to interpret:

<img src = "http://jalammar.github.io/images/t/transformer_self-attention_visualization_3.png" width = "40%">


## Representing The Order of The Sequence Using Positional Encoding

The transformer adds a vector to each input embedding. These vectors follow a specific pattern that the model learns. Which helps it determine the __position of each word, or the distance between different words in the sequence__.

The intuition here is that adding these values to the embeddings provides meaningful distances between the embedding vectors once they’re projected into Q/K/V vectors and during dot-product attention.

<img src = "http://jalammar.github.io/images/t/transformer_positional_encoding_vectors.png" width = "80%">

`To give the model a sense of the order of the words, we add positional encoding vectors -- the values of which follow a specific pattern.`

If we assumed the embedding has a dimensionality of 4, the actual positional encodings would look like this:

<img src = "http://jalammar.github.io/images/t/transformer_positional_encoding_example.png" width = "80%">

                    `A real example of positional encoding with a toy embedding size of 4`

## Pattern

In the following figure, each row corresponds the a positional encoding of a vector. So the first row would be the vector we’d add to the embedding of the first word in an input sequence. Each row contains 512 values – each with a value between 1 and -1. We’ve color-coded them so the pattern is visible.

<img src = "http://jalammar.github.io/images/t/transformer_positional_encoding_large_example.png" width = "60%">

`A real example of positional encoding for 20 words (rows) with an embedding size of 512 (columns). You can see that it appears split in half down the center. That's because the values of the left half are generated by one function (which uses sine), and the right half is generated by another function (which uses cosine). They're then concatenated to form each of the positional encoding vectors.`

This is not the only possible method for positional encoding. It, however, gives the advantage of being able to scale to unseen lengths of sequences (e.g. if our trained model is asked to translate a sentence longer than any of those in our training set).

The method shown in the paper is slightly different in that it doesn’t directly concatenate, but interweaves the two signals.

<img src = "http://jalammar.github.io/images/t/attention-is-all-you-need-positional-encoding.png" width = "60%">


## The Residuals

Each sub-layer (self-attention, ffnn) in `each encoder has a residual connection` around it, and is followed by a `layer-normalization` step.

<img src = "http://jalammar.github.io/images/t/transformer_resideual_layer_norm.png" width = "50%">

If we’re to visualize the vectors and the layer-norm operation associated with self attention, it would look like this:

<img src = "http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png" width = "50%">

This goes for the sub-layers of the decoder as well. If we’re to think of a Transformer of 2 stacked encoders and decoders, it would look something like this:

<img src = "http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png" width = "70%">


# Decoder

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder in its “encoder-decoder attention” layer which helps the decoder focus on appropriate places in the input sequence:

<img src = "http://jalammar.github.io/images/t/transformer_decoding_1.gif" width = "80%">

` Each step in the decoding phase outputs an element from the output sequence (the English translation sentence in this case).`

The following steps repeat the process until a special symbol is reached indicating the transformer decoder has completed its output. The output of each step is fed to the bottom decoder in the next time step, and the decoders bubble up their decoding results just like the encoders did. And just like we did with the encoder inputs, we embed and add positional encoding to those decoder inputs to indicate the position of each word.

<img src = "http://jalammar.github.io/images/t/transformer_decoding_2.gif" width = "80%">


## Decoder Attention Calculation

* The self attention layers in the decoder operate in a slightly different way than the one in the encoder:

* In the decoder, the self-attention layer is only allowed to attend to earlier positions in the output sequence. This is done by __masking future positions__ (setting them to `-inf`) before the softmax step in the self-attention calculation.

* The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

# The Final Linear and Softmax Layer

* `The decoder stack outputs a vector of floats. How do we turn that into a word? That’s the job of the final Linear layer which is followed by a Softmax Layer.`

* The __Linear layer__ is a simple fully connected neural network that projects the vector produced by the stack of decoders, into a much, much larger vector called a __logits vector__.

* Let’s assume that our model knows 10,000 unique English words (our model’s “output vocabulary”) that it’s learned from its training dataset. This would make the logits vector 10,000 cells wide – each cell corresponding to the score of a unique word. That is how we interpret the output of the model followed by the Linear layer.

* The __softmax layer__ then turns those scores into probabilities (all positive, all add up to 1.0). The cell with the highest probability is chosen, and the word associated with it is produced as the output for this time step.

<img src = "http://jalammar.github.io/images/t/transformer_decoder_output_softmax.png" width = "70%">

`This figure starts from the bottom with the vector produced as the output of the decoder stack. It is then turned into an output word.`


# Recap Of Training

During training, an untrained model would go through the exact same forward pass. But since we are training it on a labeled training dataset, we can compare its output with the actual correct output.

To visualize this, let’s assume our output vocabulary only contains six words(“a”, “am”, “i”, “thanks”, “student”, and “<eos>” (short for ‘end of sentence’)).
    
<img src = "http://jalammar.github.io/images/t/vocabulary.png" width = "60%">
    
`The output vocabulary of our model is created in the preprocessing phase before we even begin training.`

Once we define our output vocabulary, we can use a vector of the same width to indicate each word in our vocabulary. This also known as one-hot encoding. So for example, we can indicate the word “am” using the following vector:
    
<img src = "http://jalammar.github.io/images/t/one-hot-vocabulary-example.png" width = "60%">

`Example: one-hot encoding of our output vocabulary`

# The Loss Function

We want the output to be a probability distribution indicating the word “thanks”. But since this model is not yet trained, that’s unlikely to happen just yet.

<img src = "http://jalammar.github.io/images/t/transformer_logits_output_and_label.png" width = "50%">

`Since the model's parameters (weights) are all initialized randomly, the (untrained) model produces a probability distribution with arbitrary values for each cell/word. We can compare it with the actual output, then tweak all the model's weights using backpropagation to make the output closer to the desired output.`

How do you compare two probability distributions? We simply subtract one from the other. For more details, look at `cross-entropy` and `Kullback–Leibler divergence`.

But note that this is an oversimplified example. More realistically, we’ll use a sentence longer than one word. For example – input: “je suis étudiant” and expected output: “i am a student”. What this really means, is that we want our model to successively output probability distributions where:

* Each probability distribution is represented by a vector of width vocab_size (6 in our toy example, but more realistically a number like 30,000 or 50,000)
* The first probability distribution has the highest probability at the cell associated with the word “i”
* The second probability distribution has the highest probability at the cell associated with the word “am”
* And so on, until the fifth output distribution indicates [end of sentence] symbol, which also has a cell associated with it from the 10,000 element vocabulary.
    
<img src = "http://jalammar.github.io/images/t/output_target_probability_distributions.png" width = "60%">
    
`The targeted probability distributions we'll train our model against in the training example for one sample sentence.`
    
After training the model for enough time on a large enough dataset, we would hope the produced probability distributions would look like this:
    
<img src = "http://jalammar.github.io/images/t/output_trained_model_probability_distributions.png" width = "60%">

`Hopefully upon training, the model would output the right translation we expect. Of course it's no real indication if this phrase was part of the training dataset (see: cross validation). Notice that every position gets a little bit of probability even if it's unlikely to be the output of that time step -- that's a very useful property of softmax which helps the training process.`

Now, because the model produces the outputs one at a time, we can assume that the model is selecting the word with the __highest probability__ from that probability distribution and throwing away the rest. That’s one way to do it (__called greedy decoding__). 

Another way to do it would be to hold on to, say, the __top two words__ (say, ‘I’ and ‘a’ for example), then in the next step, __run the model twice__: once assuming the first output position was the word ‘I’, and another time assuming the first output position was the word ‘a’, and whichever version produced less error considering both positions #1 and #2 is kept. We repeat this for positions #2 and #3…etc. This method is called __“beam search”__, where in our example, beam_size was two (meaning that at all times, two partial hypotheses (unfinished translations) are kept in memory), and top_beams is also two (meaning we’ll return two translations). These are both hyperparameters that you can experiment with.
